In [4]:
import os
import sys

from models.model import FER2013CNN

In [5]:
#main imports

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from torchsummary import summary

from tqdm import tqdm
import cv2

In [6]:
from data.dataloader import load_dataloader

In [7]:
print(torch.backends.mps.is_available())

True


In [8]:
def train_model(model, train_loader, criterion, optimizer, epochs, val_loader=None):

    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model = model.to(device)
    
    train_accuracies = []
    val_accuracies = []
    
    for epoch in range(1, epochs + 1):
        model.train()  
        epoch_loss = 0.0
        
        correct, total = 0, 0
        
        with tqdm(train_loader, unit="batch", desc=f"Epoch {epoch}/{epochs}") as pbar:
            for images, labels in pbar:

                images = images.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()  # Reset gradients
                outputs = model(images)  # Feed images to model
                loss = criterion(outputs, labels)
                loss.backward()  # Get gradients
                optimizer.step()  # Apply gradients

                epoch_loss += loss.item()
                _, prediction = torch.max(outputs, 1)
                correct += (prediction == labels).sum().item()
                total += labels.size(0)

                # Update the tqdm description (optional)
                pbar.set_postfix(loss=epoch_loss / total, accuracy=100 * correct / total)

        if epoch % 5 == 0 and val_loader:
            model.eval()
            val_loss = 0.0
            val_correct = 0.0
            val_total = 0.0

            with torch.no_grad():     
                 with tqdm(val_loader, unit="batch", desc=f"Validation Round {epoch // 5}") as vpbar:
                    for images, labels in vpbar:
                        images = images.to(device)
                        labels = labels.to(device)
                        outputs = model(images)
                        loss = criterion(outputs, labels)
    
                        val_loss += loss.item()
                        _, prediction = torch.max(outputs, 1)
                        val_correct += (prediction == labels).sum().item()
                        val_total += labels.size(0)

                        val_acc = 100 * val_correct / val_total
                        vpbar.set_postfix(loss=val_loss / val_total, accuracy=val_acc)
            val_accuracies.append(val_acc)
        train_acc = 100 * correct / total
        train_accuracies.append(train_acc)
        print(f"Epoch {epoch} : Train Accuracy = {train_acc:.2f}%, Loss = {epoch_loss:.4f}")
    
    return model, train_accuracies, val_accuracies

In [9]:
cnn = FER2013CNN(num_classes=7)

In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
optimizer = optim.Adam(cnn.parameters(), lr=1e-3)

In [12]:
train_transform = transforms.Compose([
    transforms.ToTensor(),  
    transforms.RandomHorizontalFlip(),  
    transforms.RandomRotation(10),
    transforms.Normalize(mean=[0.5], std=[0.5])  
])

val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [13]:
train_loader = load_dataloader("data/dataset/fer2013.csv", 'train', 32, transform=train_transform)
val_loader = load_dataloader("data/dataset/fer2013.csv", 'val', 32, transform=val_transform)

In [14]:
for images, labels in train_loader:
    assert images.size() == torch.Size([32, 1, 48, 48])
    assert labels.dtype == torch.int64
    print("test passed!")
    break

for images, labels in val_loader:
    assert images.size() == torch.Size([32, 1, 48, 48])
    assert labels.dtype == torch.int64
    print("test passed!")
    break

test passed!
test passed!


In [15]:
model, cnn_trained, train_accuracies = train_model(
    model=cnn,
    train_loader=train_loader,
    criterion=criterion,
    optimizer=optimizer,
    epochs=70,
    val_loader=val_loader
)

Epoch 1/70: 100%|█████████████████████████████████████████████████████████████████████████| 898/898 [00:16<00:00, 53.69batch/s, accuracy=29.2, loss=0.0547]


Epoch 1 : Train Accuracy = 29.17%, Loss = 1570.3833


Epoch 2/70: 100%|█████████████████████████████████████████████████████████████████████████████| 898/898 [00:16<00:00, 55.05batch/s, accuracy=36, loss=0.05]


Epoch 2 : Train Accuracy = 35.95%, Loss = 1436.7755


Epoch 3/70: 100%|█████████████████████████████████████████████████████████████████████████| 898/898 [00:16<00:00, 52.89batch/s, accuracy=38.6, loss=0.0482]


...

Epoch 68 : Train Accuracy = 64.54%, Loss = 835.8840


Epoch 69/70: 100%|█████████████████████████████████████████████████████████████████████████| 898/898 [00:09<00:00, 90.97batch/s, accuracy=64.4, loss=0.029]


Epoch 69 : Train Accuracy = 64.42%, Loss = 833.4870


Validation Round 14: 100%|███████████████████████████████████████████████████████████████| 113/113 [00:00<00:00, 278.36batch/s, accuracy=60.7, loss=0.0364]

Epoch 70 : Train Accuracy = 64.87%, Loss = 831.1558


In [16]:
EMOTION_LABELS = {
    0: 'Angry',
    1: 'Disgust',
    2: 'Fear',
    3: 'Happy',
    4: 'Sad',
    5: 'Surprise',
    6: 'Neutral'
}

In [18]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(device)

EMOTION_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Webcam could not be opened.")
    exit()

try:
    while True:
        start_time = cv2.getTickCount()

        ret, frame = cap.read()
        if not ret:
            print("Could not read frame.")
            continue

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            face = gray[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (48, 48))
            face_normalized = face_resized.astype('float32') / 255.0
            face_tensor = torch.tensor(face_normalized).unsqueeze(0).unsqueeze(0).float().to(device)
            assert face_tensor.size() == torch.Size([1, 1, 48, 48])

            model.eval()
            with torch.no_grad():
                outputs = model(face_tensor)
                prob, predicted = torch.max(outputs, 1)
                emotion = EMOTION_LABELS[predicted.item()]

            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{emotion} : {prob.item()}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 
                        0.6, (0, 255, 0), 2, cv2.LINE_AA)

        fps = cv2.getTickFrequency() / (cv2.getTickCount() - start_time)
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                    0.7, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow("Emotion Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    cap.release()
    cv2.destroyAllWindows()


2025-05-17 10:02:31.255 Python[92160:7045226] _TIPropertyValueIsValid called with 16 on nil context!
2025-05-17 10:02:31.255 Python[92160:7045226] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-05-17 10:02:31.255 Python[92160:7045226] Text input context does not respond to _valueForTIProperty:
2025-05-17 10:02:32.239 Python[92160:7045226] _TIPropertyValueIsValid called with 16 on nil context!
2025-05-17 10:02:32.239 Python[92160:7045226] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-05-17 10:02:32.239 Python[92160:7045226] Text input context does not respond to _valueForTIProperty:


In [3]:
model

NameError: name 'model' is not defined